# Western Governors University
## D211 - Advanced Data Acquisition
### Shane Boyce

In [17]:
import pandas as pd

#### Data Preparation

In order to normalize wa_fn as much as possible to make visualization easier, each CSV will be read into a pandas dataframe to assist with cleaning wa_fn before loading it into the Postgres database. The following code will read each CSV into a dataframe and then print the first 5 rows of each dataframe to verify the data was read correctly.


In [18]:
#unclean and unnormalized data to be read into pandas and then uploaded to Postgres
wa_fn = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
wa_fn.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [19]:
#normalized tables for referencing
services = pd.read_csv('services.csv')
contract = pd.read_csv('contract.csv')
customer = pd.read_csv('customer.csv')
payment = pd.read_csv('payment.csv', index_col=None)
surveys = pd.read_csv('Survey_Responses.csv')

In [20]:
#verify tables were read correctly
services.head()

,customer_id,InternetService,Phone,Multiple,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport
0,A00088,Fiber Optic,Yes,Yes,Yes,No,No,No
1,A04204,DSL,Yes,Yes,Yes,Yes,Yes,Yes
2,A04378,None,Yes,Yes,No,No,No,Yes
3,A04830,DSL,Yes,Yes,Yes,No,Yes,No
4,A05946,Fiber Optic,No,No,No,No,No,No


In [21]:
payment.head()

,payment_id,payment_type
0,1,Bank Transfer Automatic
1,2,Credit Card Automatic
2,3,Electronic Check
3,4,Mailed Check


In [22]:
customer.head()

,customer_id,lat,lng,population,children,age,income,marital,churn,gender,...,email,contacts,yearly_equip_faiure,techie,port_modem,tablet,job_id,payment_id,contract_id,location_id
0,K409198,56.25100,-133.37571,38,0,68,28561.99,Widowed,No,Male,...,10,0,1,No,Yes,Yes,229,2,2,5599
1,S120509,44.32893,-84.24080,10446,1,27,21704.77,Married,Yes,Female,...,12,0,1,Yes,No,Yes,468,1,1,2737
2,K191035,45.35589,-123.24657,3735,4,50,9609.57,Widowed,No,Female,...,9,0,1,Yes,Yes,No,96,2,3,1297
3,D90850,32.96687,-117.24798,13863,1,48,18925.23,Married,No,Male,...,15,2,0,Yes,No,No,552,4,3,5181
4,K662701,29.38012,-95.80673,11352,0,83,40074.19,Separated,Yes,Male,...,16,2,1,No,Yes,No,371,4,1,30


In [23]:
#While this wasn't read in properly by pandas, it is a reference only to assist with cleaning wa_fn and will not be updated here.
contract.head()

,1,Month-to-month
0,2,One year
1,3,Two Year


In [24]:
surveys.head()

,customer_id,Timely_Responses,Timely Fixes,Timely_Replacement,Reliability,Options,Respectful,Courteous,Active_Listening
0,A00088,2,3,3,2,6,2,2,4
1,A04204,5,3,4,1,6,3,3,2
2,A04378,4,5,4,2,4,4,2,3
3,A04830,3,3,3,5,3,4,5,4
4,A05946,3,3,3,5,3,4,4,4


In [25]:
#view columns in wa_fn to assist with cleaning
wa_fn.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [26]:
customer.columns

Index(['customer_id', 'lat', 'lng', 'population', 'children', 'age', 'income',
       'marital', 'churn', 'gender', 'tenure', 'monthly_charge',
       'bandwidth_gp_year', 'outage_sec_week', 'email', 'contacts',
       'yearly_equip_faiure', 'techie', 'port_modem', 'tablet', 'job_id',
       'payment_id', 'contract_id', 'location_id'],
      dtype='object')

In [27]:
wafncols = wa_fn.columns.tolist()
cuscols = customer.columns.tolist()
serv_cols = services.columns.tolist()
rename_dict = {}
wafncols, cuscols, serv_cols

(['customerID',
  'gender',
  'SeniorCitizen',
  'Partner',
  'Dependents',
  'tenure',
  'PhoneService',
  'MultipleLines',
  'InternetService',
  'OnlineSecurity',
  'OnlineBackup',
  'DeviceProtection',
  'TechSupport',
  'StreamingTV',
  'StreamingMovies',
  'Contract',
  'PaperlessBilling',
  'PaymentMethod',
  'MonthlyCharges',
  'TotalCharges',
  'Churn'],
 ['customer_id',
  'lat',
  'lng',
  'population',
  'children',
  'age',
  'income',
  'marital',
  'churn',
  'gender',
  'tenure',
  'monthly_charge',
  'bandwidth_gp_year',
  'outage_sec_week',
  'email',
  'contacts',
  'yearly_equip_faiure',
  'techie',
  'port_modem',
  'tablet',
  'job_id',
  'payment_id',
  'contract_id',
  'location_id'],
 ['customer_id',
  'InternetService',
  'Phone',
  'Multiple',
  'OnlineSecurity',
  'OnlineBackup',
  'DeviceProtection',
  'TechSupport'])

In [28]:
for col in cuscols:
    for col2 in wafncols:
        if col[:5] == col2[:5].lower():
            rename_dict[col2] = col
        else:
            pass


rename_dict


{'customerID': 'customer_id',
 'Churn': 'churn',
 'gender': 'gender',
 'tenure': 'tenure',
 'MonthlyCharges': 'monthly_charge',
 'PaymentMethod': 'payment_id',
 'Contract': 'contract_id'}

In [32]:
#rename customerID to customer_id to match the customer table
wa_fn.rename(columns=rename_dict, inplace=True)
wa_fn.head(1).transpose()some

,0
customer_id,7590-VHVEG
gender,Female
SeniorCitizen,0
Partner,Yes
Dependents,No
tenure,1
PhoneService,No
MultipleLines,No phone service
InternetService,DSL
OnlineSecurity,No


####Part 1:  Data Dashboards

A.  Provide a copy of your dashboards that support executive decision-making.

A Tableau workbook is provided within this repository.

1.  Provide both data sets that serve as the data source for the dashboards.

The datasets are included here as CSVs (A normalized `Customer` as 7 csvs dataset with an non-normalized `wa_fn` dataset).

2.  Provide step-by-step instructions to guide users through the dashboard installation.

This dashboarding experience requires the attached CSV files, a local instance of PostgreSql with the ability to use COPY in the DML query tool or \copy in PSQL command line environment. In part 4, the Psycopg2 libary is used to connect to the database and execute the COPY command. The attached CSVs are used to create the tables in the database. The attached Tableau workbook is used to connect to the database and create the dashboards.

3.  Provide clear instructions to help users navigate the dashboards.

4.  Provide a copy of all SQL code and other code supporting the dashboards.

In [30]:
# DDL statements to create tables in the database



#### Part 2:  Demonstration

A video walkthrough was included via WGU's internal Panapto tool covering the following:

B.

1.  Describe the technical environment used to create the dashboards.

2.  Demonstrate the functionality of the dashboards.

3.  Explain the SQL scripts used to support the creation of the dashboards.

4.  Explain how the data streams were prepared to support the analysis.

5.  Describe how data were aligned with other data points.

6.  Demonstrate how the databases were created.

7.  Explain how referential integrity was enforced in the database.

#### Part 3: Reflection

Part 3:  Report

C.  Write a report to outline the data exploration, use of advanced SQL operations, and the analysis of the data. Do the following as part of your report:

1.  Explain how the purpose and function of your dashboard aligns with the needs outlined in the data dictionary associated with your chosen data set.

2.  Justify the selection of the business intelligence tool you used.

3.  Explain the steps used to clean and prepare the data for the analysis.

4.  Summarize the steps used to create the dashboards.

5.  Discuss the results of your data analysis and how it supports executive decision-making.

6.  Discuss the limitation(s) of your data analysis.